In [ ]:
#Modules required roslaunch, rospy, time, numpy, std_msgs, tf2_msgs, tf

In [ ]:
%matplotlib inline

import roslaunch, rospy, time, keyboard
import rospkg as rp
import numpy as np
from std_msgs.msg import Float64
from tf2_msgs.msg import TFMessage
from tf.transformations import euler_from_quaternion
import matplotlib.pyplot as plt
import time, threading

from collections import deque

pi = 3.1415926

f_topic = '/twolinkman/front_joint_effort_controller/command'
r_topic = '/twolinkman/rear_joint_effort_controller/command'
rcon_launch_path = '/launch/twolinkman_control.launch'
rp = rp.RosPack()
package_path = rp.get_path('twolinkman')

class Joint:
	def __init__(self,topic,setpoint):
		self.topic = topic
		self.pub = setpoint

class PID:
    e = 0
    E = 0
    ep = 0
    def __init__(self,kp = 0,ki = 0,kd = 0):
        self.kp = kp
        self.ki = ki
        self.kd = kd
    def pid(self, setpoint, process_value):
        self.e = setpoint-process_value
        self.E = self.E + self.e
        ed = self.e - self.ep
        self.ep = self.e
        return (self.kp*self.e+self.kd*ed+self.ki*self.E)    

In [ ]:
#ROS Control Launch file
rcon_uuid = roslaunch.rlutil.get_or_generate_uuid(None, False)
roslaunch.configure_logging(rcon_uuid)
rcon_launch = roslaunch.parent.ROSLaunchParent(rcon_uuid, [package_path+rcon_launch_path])
rcon_launch.start()

In [ ]:
#Environment ready-->
#SetPoint Node
front_joint = Joint(f_topic,None)
rear_joint = Joint(r_topic,None)
front_PID = PID(0.1,0.0,0)
rear_PID = PID(0.1,0.0,0)

def listener_callback(data):
    global front_angle, rear_angle
    quatf = data.transforms[0].transform.rotation
    quatr = data.transforms[1].transform.rotation
    (front_angle, _, _)= euler_from_quaternion([quatf.x,quatf.y,quatf.z,quatf.w])
    (rear_angle, _, _)= euler_from_quaternion([quatr.x,quatr.y,quatr.z,quatr.w])
    front_angle = front_angle*180/pi
    rear_angle = rear_angle*180/pi
    #rospy.loginfo('Front angle: '+str(front_angle)+' Rear angle: '+str(rear_angle))    

setpointnode = rospy.init_node('SetPoint',anonymous=True)
front_joint.pub = rospy.topics.Publisher(front_joint.topic,Float64,queue_size=10)
rear_joint.pub = rospy.topics.Publisher(rear_joint.topic,Float64,queue_size=10)
listener = rospy.topics.Subscriber('/tf',TFMessage,callback=listener_callback,queue_size = 10)



In [ ]:
plt_front_angle = deque([]);
plt_rear_angle = deque([]);

fig = plt.figure();
ax1 = fig.add_subplot(1,1,1);
ax1.set_xlim([0, 20]);
ax1.set_ylim([-90, 90]);

def plot_data():
    global plt_front_angle
    frontangle = plt_front_angle
    ax1.set_xlim([0, 20]);
    ax1.set_ylim([-90, 90]);
    ax1.clear();
    x = [i for i in range(len(frontangle))]
    ax1.plot(x,frontangle);
    plt.show()
    threading.Timer(0.5, plot_data).start()

#plot_data();
while not rospy.is_shutdown():
    time.sleep(0.05);
    front_joint.pub.publish(front_PID.pid(-45,front_angle));
    rear_joint.pub.publish(rear_PID.pid(35,rear_angle));
    if(len(plt_front_angle) < 20) :
        plt_front_angle.append(front_angle);
    else:
        plt_front_angle.rotate(-1);
        plt_front_angle[19] = front_angle;
#    ax1.clear()
    ax1.plot([i for i in range(len(plt_front_angle))],plt_front_angle);
#    plt.pause(1)
#	print(published_topics)
#gz_launch.shutdown()
#rcon_launch.shutdown()
	

In [ ]:
print(x)
print(plt_front_angle)
fig = plt.figure()
ax = fig.add_subplot(111) 
ax.plot(x,plt_front_angle,'r-');

In [ ]:
x = [i for i in range(len(plt_front_angle))]
print(len(x))
print(len(plt_front_angle))

In [ ]:
'''
def norm(var1, var2):
    norm = 0
    if len(var1) == len(var2):
        for i in range(len(var1)):
            norm = norm+(var1[i]-var2[i])**2
    else:
        print('Invalid inputs')
        norm = -1
    return norm
'''

In [ ]:
#Computed Torque Controller
a1 = 1
a2 = 1
m1 = 1
m2 = 1
th1 = 0 
thd1 = 0
th2 = 0 
thd2 = 0
g = 9.8
D = np.matrix([[(m1+m2)*a1**2+m2*a2**2+2*m2*a1*a2*np.cos(th2), m2*a2**2+m2*a1*a2*np.cos(th2)], [m2*a2**2+m2*a1*a2*np.cos(th2), m2*a2**2]])
H = np.matrix([[-m2*a1*a2*(2*thd1*thd2+thd2**2)*np.sin(th2)], [m2*a1*a2*thd1**2*np.sin(th2)]])
C = np.matrix([[(m1+m2)*a1*g*np.cos(th1)+m2*a2*g*np.cos(th1+th2)], [m2*a2*g*np.cos(th1+th2)]])
#T = D*(thdd-u)+H+C
print(D)
print(H)
print(C)